In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.classification.LogisticRegression

import org.apache.spark.sql.functions._ 

val spark = SparkSession.builder().getOrCreate()


val df = spark.read.format("csv").option("header", "true").load("raw_files/features.csv")
val dfs = df.drop(df.col("_c0"))
dfs.show(5)






Intitializing Scala interpreter ...

Spark Web UI available at http://vpn-082-252.hs-ulm.de:4040
SparkContext available as 'sc' (version = 2.4.0, master = local[*], app id = local-1554832590601)
SparkSession available as 'spark'


+---------------+-------------+---------+-----------------+---------+------+-----------+-------+-----+----------+----+------------+-------------+
|StartStation Id|EndStation Id|Frequency|    Daily Weather|  Weekday|Season|Day & Night|Holiday|Month|      Date|Year|Passed_Years|Passed_Months|
+---------------+-------------+---------+-----------------+---------+------+-----------+-------+-----+----------+----+------------+-------------+
|            836|          749|        1|partly-cloudy-day| Thursday|Autumn|      night|  False|    9|2018-09-06|2018|           1|            7|
|            831|          638|        5|        clear-day|  Tuesday|Summer|      night|  False|    7|2018-07-24|2018|           1|            9|
|            831|          264|        1|partly-cloudy-day|Wednesday|Summer|        day|  False|    6|2018-06-06|2018|           1|           10|
|            831|          372|        2|        clear-day|Wednesday|Spring|        day|  False|    4|2018-04-18|2018|      

import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.sql.functions._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@707e4ccd
df: org.apache.spark.sql.DataFrame = [_c0: string, StartStation Id: string ... 12 more fields]
dfs: org.apache.spark.sql.DataFrame = [StartStation Id: string, EndStation Id: string ... 11 more fields]


In [2]:
import org.apache.spark.sql.types._

val Array(training, test) = dfs.randomSplit(Array[Double](0.7, 0.3), 18)

val training_df = (training.select(training("Frequency").as("label"),$"Daily Weather",$"Day & Night" )).withColumn("label", 'label cast DoubleType)
val train_set = training_df.na.drop()




import org.apache.spark.sql.types._
training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [StartStation Id: string, EndStation Id: string ... 11 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [StartStation Id: string, EndStation Id: string ... 11 more fields]
training_df: org.apache.spark.sql.DataFrame = [label: double, Daily Weather: string ... 1 more field]
train_set: org.apache.spark.sql.DataFrame = [label: double, Daily Weather: string ... 1 more field]


In [3]:
train_set.show(5)

+-----+-----------------+-----------+
|label|    Daily Weather|Day & Night|
+-----+-----------------+-----------+
|  3.0|partly-cloudy-day|      night|
| 19.0|partly-cloudy-day|      night|
|  8.0|partly-cloudy-day|        day|
|  3.0|partly-cloudy-day|      night|
|228.0|partly-cloudy-day|        day|
+-----+-----------------+-----------+
only showing top 5 rows



In [5]:
import org.apache.spark.ml.feature.{OneHotEncoder, VectorIndexer, VectorAssembler, StringIndexer}
import org.apache.spark.ml.Pipeline

// converting strings into numerical values
//val FreqIndexer = new StringIndexer().setInputCol("Frequency").setOutputCol("Freq")

val DayPerIndexer = new StringIndexer().setInputCol("Day & Night").setOutputCol("DayPer")
val DayWthIndexer = new StringIndexer().setInputCol("Daily Weather").setOutputCol("DayWth")

val LabelIndexer = new StringIndexer().setInputCol("label").setOutputCol("DayWth")

//val indexed = indexer.transform(df)

// converting numerical values into one hot encoding (0 or 1)
//val FreqEncoder = new OneHotEncoder().setInputCol("Freq").setOutputCol("FreqVec")
val DayPerEncoder = new OneHotEncoder().setInputCol("DayPer").setOutputCol("DayPerVec")
val DayWthEncoder = new OneHotEncoder().setInputCol("DayWth").setOutputCol("DayWthVec")

// (Label, features)
var assembler= (new VectorAssembler()
                .setInputCols(Array( "DayWthVec", "DayPerVec")).setOutputCol("features"))




//val encoded = encoder.transform(indexed)
//encoded.show()

import org.apache.spark.ml.feature.{OneHotEncoder, VectorIndexer, VectorAssembler, StringIndexer}
import org.apache.spark.ml.Pipeline
DayPerIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_20772f3ff977
DayWthIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_228bb5e0ae81
LabelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_0f6008498d7a
DayPerEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHot_9cda62599b4b
DayWthEncoder: org.apache.spark.ml.feature.OneHotEncoder = oneHot_af755f2e23e6
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_f18b7c236ab8


In [6]:
val lr = new LogisticRegression()
  .setMaxIter(10)
  .setRegParam(0.3)
  .setElasticNetParam(0.8)

val pipeline = new Pipeline().setStages(Array( DayPerIndexer,DayPerEncoder, DayWthIndexer, DayWthEncoder, assembler))
//val pipeline = new Pipeline().setStages(Array( DayPerIndexer,DayPerEncoder, DayWthIndexer, DayWthEncoder, assembler, lr))


lr: org.apache.spark.ml.classification.LogisticRegression = logreg_1aa307bf02f8
pipeline: org.apache.spark.ml.Pipeline = pipeline_de511f9d4751


In [8]:
//val model = pipeline.fit(train_set)
var pipe=pipeline.fit(train_set).transform(train_set)
pipe.show

+-----+-----------------+-----------+------+-------------+------+-------------+-------------------+
|label|    Daily Weather|Day & Night|DayPer|    DayPerVec|DayWth|    DayWthVec|           features|
+-----+-----------------+-----------+------+-------------+------+-------------+-------------------+
|  3.0|partly-cloudy-day|      night|   1.0|    (1,[],[])|   0.0|(6,[0],[1.0])|      (7,[0],[1.0])|
| 19.0|partly-cloudy-day|      night|   1.0|    (1,[],[])|   0.0|(6,[0],[1.0])|      (7,[0],[1.0])|
|  8.0|partly-cloudy-day|        day|   0.0|(1,[0],[1.0])|   0.0|(6,[0],[1.0])|(7,[0,6],[1.0,1.0])|
|  3.0|partly-cloudy-day|      night|   1.0|    (1,[],[])|   0.0|(6,[0],[1.0])|      (7,[0],[1.0])|
|228.0|partly-cloudy-day|        day|   0.0|(1,[0],[1.0])|   0.0|(6,[0],[1.0])|(7,[0,6],[1.0,1.0])|
|  1.0|        clear-day|        day|   0.0|(1,[0],[1.0])|   1.0|(6,[1],[1.0])|(7,[1,6],[1.0,1.0])|
| 11.0|        clear-day|        day|   0.0|(1,[0],[1.0])|   1.0|(6,[1],[1.0])|(7,[1,6],[1.0,1.0])|


pipe: org.apache.spark.sql.DataFrame = [label: double, Daily Weather: string ... 6 more fields]


In [ ]:
//val ss = (training.select(training("Frequency").as("label"),$"Daily Weather",$"Day & Night" )).withColumn("label", 'label cast DoubleType)

//val model = pipeline.fit(train_set)
//val results= model.transform(test)
//var s= pipe.drop(pipe.col("Day & Night","Daily Weather"))
//s= s.drop(s.col("Daily Weather"))
//s= s.drop(s.col("DayPer"))
//s= s.drop(s.col("DayWth"))

//s.show


In [ ]:

val lrModel = lr.fit(pipe)


